<a href="https://colab.research.google.com/github/gabriel7268/ENACOMP_2020/blob/main/ENACOMP_Minicurso_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving PETR4.SA - Teste.csv to PETR4.SA - Teste (1).csv
Saving PETR4.SA - Treinamento.csv to PETR4.SA - Treinamento (1).csv


In [ ]:
import io
base = pd.read_csv(io.BytesIO(uploaded['PETR4.SA - Treinamento.csv']))
#exclui valores nulos
base = base.dropna()
#controi uma nova struct com apenas dados da coluna open
base_treinamento = base.iloc[:,1:2].values
#normaliza os dados em intervalos de 0 e 1
normalizador = MinMaxScaler(feature_range=[0,1])
base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

previsores = []
preco_real = []

#90 dias anteriores
for i in range(90,1225):
  previsores.append(base_treinamento_normalizada[i-90:i,0])
  preco_real.append(base_treinamento_normalizada[i,0])

#transforma para os dados ficarem no formato do numpy
previsores, preco_real = np.array(previsores), np.array(preco_real)

previsores = np.reshape(previsores, (previsores.shape[0], previsores.shape[1], 1))

regressor = Sequential()
#camada 1
regressor.add(LSTM(units = 100, return_sequences = True, input_shape = (previsores.shape[1], 1)))

regressor.add(Dropout(0.3))

#camada 2
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

#camada 3
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

#camada 4
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

#camada saida
regressor.add(Dense(units = 1, activation = 'linear'))

regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error',
                  metrics = ['mean_absolute_error'])

regressor.fit(previsores, preco_real, epochs = 100, batch_size = 32)

base_teste = pd.read_csv(io.BytesIO(uploaded['PETR4.SA - Teste.csv']))
base_teste = base_teste.dropna()
preco_real_teste = base_teste.iloc[:,1:2]].values
base_completa = pd.concat((base['Open'], base_teste['Open']), axis = 0)

entradas = base_completa[len(base_completa)-len(base_teste)-90:].values

entradas = entradas.reshape(-1,1)

entradas = normalizador.transform(entradas)

X_teste = []
for i in range (90, 313):
  X_teste.append(entradas[i-90:i,0])
X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))
previsoes = regressor.predict(X_teste)
previsoes = normalizador.inverse_transform(previsoes)

previsoes.mean()
preco_real_teste.mean()

plt.plot(preco_real_teste, color = 'red', label = 'Preço real')
plt.plot(previsoes, color = 'blue', label = 'Previsões')
plt.title('Previsão preço das ações')
plt.xlabel('Tempo')
plt.ylabel('Valor Yahoo')
plt.legend()
plt.show()